### This notebook contains contains code for Task 3 (All-language fine tuning & testing)

In [ ]:
!pip install -r "requirements.txt"

In [2]:
import torch
import math
from torch import nn, dtype
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.utils.data import random_split, DataLoader, Dataset
from config import PretrainedConfig, GPT2Config
from transformers import GPT2Model as OpenAIGPT2Model
from transformers import GPT2Tokenizer
from utils import *
from einops import rearrange
from typing import Callable, Iterable, Tuple
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


## Model architecture
Different from the official GPT-2 architecture; this one is without the language modelling head.

In [3]:
class CausalSelfAttention(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.num_attention_heads = config.num_attention_heads
    self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
    self.all_head_size = self.num_attention_heads * self.attention_head_size

    # Initialize the linear transformation layers for key, value, query.
    self.query = nn.Linear(config.hidden_size, self.all_head_size)
    self.key = nn.Linear(config.hidden_size, self.all_head_size)
    self.value = nn.Linear(config.hidden_size, self.all_head_size)
    # This dropout is applied to normalized attention scores following the original
    # implementation of transformer. Although it is a bit unusual, we empirically
    # observe that it yields better performance.
    self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

  def transform(self, x, linear_layer):
    # The corresponding linear_layer of k, v, q are used to project the hidden_state (x).
    proj = linear_layer(x)
    # Next, we need to produce multiple heads for the proj. This is done by spliting the
    # hidden state to self.num_attention_heads, each of size self.attention_head_size.
    proj = rearrange(proj, 'b t (h d) -> b t h d', h=self.num_attention_heads)
    # By proper transpose, we have proj of size [bs, num_attention_heads, seq_len, attention_head_size].
    proj = rearrange(proj, 'b t h d -> b h t d')
    return proj

  def attention(self, key, query, value, attention_mask):
    """
    TODO-1: Compute scaled dot-product attention for GPT-2.

    Implementation hints:
    1. Compute raw attention scores using QK^T, and scale them by sqrt(d_k).
    2. Apply a causal mask (lower-triangular) to prevent attending to future tokens.
    3. Optionally add the external attention_mask (e.g., padding positions).
    4. Normalize the scores with softmax to obtain attention probabilities.
    5. Apply dropout on the probabilities.
    6. Use them to weight the values (V) and obtain the context vectors.
    7. Finally, merge all attention heads back into a single hidden representation.
    """
    ### YOUR CODE HERE
    #query=(num_queries,d_k)
    #compute raw attention scores
    attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.attention_head_size)
    #apply causal mask
    #create the lower triangular matrix
    seq_len = attention_scores.size(-1)
    causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=attention_scores.device))
    causal_mask = causal_mask.unsqueeze(0).unsqueeze(0) #add batch and head dimensions
    #apply mask, set positions with 0 to -inf
    attention_scores = attention_scores.masked_fill(causal_mask == 0, -1e9)
    #add external attention mask
    if attention_mask is not None:
      attention_scores = attention_scores + attention_mask
    #normalize scores with softmax
    attention_probs = torch.softmax(attention_scores, dim=-1)
    #apply dropout
    attention_probs = self.dropout(attention_probs)
    #weigh the values(attention_prob@value)
    context_vector=torch.matmul(attention_probs,value)
    #merge all attention heads into 1
    batch_size, num_heads, seq_len, head_size = context_vector.size()
    context = context_vector.transpose(1, 2).contiguous().reshape(batch_size, seq_len, num_heads * head_size)


    return context


  def forward(self, hidden_states, attention_mask):
    """
    hidden_states: [bs, seq_len, hidden_state]
    attention_mask: [bs, 1, 1, seq_len]
    output: [bs, seq_len, hidden_state]
    """
    # First, we have to generate the key, value, query for each token for multi-head attention
    # using self.transform (more details inside the function).
    # Size of *_layer is [bs, num_attention_heads, seq_len, attention_head_size].
    key_layer = self.transform(hidden_states, self.key)
    value_layer = self.transform(hidden_states, self.value)
    query_layer = self.transform(hidden_states, self.query)
    
    # Calculate the multi-head attention using the self.attention function.
    attn_value = self.attention(key_layer, query_layer, value_layer, attention_mask)
    return attn_value

In [4]:
class GPT2Layer(nn.Module):
  def __init__(self, config):
    super().__init__()
    # Multi-head attention.
    self.self_attention = CausalSelfAttention(config)
    # Add-norm for multi-head attention.
    self.attention_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.attention_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.attention_dropout = nn.Dropout(config.hidden_dropout_prob)
    # Feed forward.
    self.interm_dense = nn.Linear(config.hidden_size, config.intermediate_size)
    self.interm_af = F.gelu
    # Add-norm for feed forward.
    self.out_dense = nn.Linear(config.intermediate_size, config.hidden_size)
    self.out_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.out_dropout = nn.Dropout(config.hidden_dropout_prob)

  def add(self, input, output, dense_layer, dropout):
    """
    TODO-2: Residual connection with dense projection and dropout.
    
    Implementation hints:
    1. Project the 'output' through dense_layer.
    2. Apply dropout to prevent overfitting.
    3. Add the original 'input' (residual connection) to the processed output.
    """

    ### YOUR CODE HERE
    #project output through dense_layer
    projected=dense_layer(output)
    #apply dropout
    projected=dropout(projected)
    #add OG input to output
    return input+projected


  def forward(self, hidden_states, attention_mask):
    """
    TODO-3: Forward pass of a GPT-2 layer.

    Implementation hints:
    ---- Self-Attention Block ----
    1. LayerNorm the input for stability using self.attention_layer_norm.
    2. Compute multi-head causal self-attention using self.self_attention.
    3. Apply residual connection using self.add (dense_layer=self.attention_dense, dropout=self.attention_dropout).

    ---- Feed Forward Block ----
    4. LayerNorm the hidden_states from attention block using self.out_layer_norm.
    5. Pass through a two-layer feed-forward network with activation:
       self.interm_dense -> self.interm_af -> self.out_dense
    6. Apply residual connection again using self.add (dense_layer=self.out_dense, dropout=self.out_dropout).
    """
    ### YOUR CODE HERE
    #1. layernorm the input
    normed_input=self.attention_layer_norm(hidden_states)
    #2. compute the self attention
    attn_output=self.self_attention(normed_input,attention_mask)
    #3. apply residual connection
    hidden_states=self.add(hidden_states,attn_output,self.attention_dense,self.attention_dropout)
    #4. layernorm the hidden states from attention blocks
    normed_ff=self.out_layer_norm(hidden_states)
    #5. pass through 2 layer feed forward
    ff_output=self.interm_dense(normed_ff)
    ff_output=self.interm_af(ff_output)
    ff_output = self.out_dense(ff_output)
    ff_output = self.out_dropout(ff_output)
    #6. apply residual connection
    hidden_states = hidden_states + ff_output
    return hidden_states

In [5]:
class GPTPreTrainedModel(nn.Module):

  def __init__(self, config: PretrainedConfig, *inputs, **kwargs):
    super().__init__()
    self.config = config
    self.name_or_path = config.name_or_path

  def init_weights(self):
    # Initialize weights
    self.apply(self._init_weights)

  def _init_weights(self, module):
    """ Initialize the weights """
    if isinstance(module, (nn.Linear, nn.Embedding)):
      # Slightly different from the TF version which uses truncated_normal for initialization
      # cf https://github.com/pytorch/pytorch/pull/5617
      module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
    elif isinstance(module, nn.LayerNorm):
      module.bias.data.zero_()
      module.weight.data.fill_(1.0)
    if isinstance(module, nn.Linear) and module.bias is not None:
      module.bias.data.zero_()

  @property
  def dtype(self) -> dtype:
    return get_parameter_dtype(self)

In [6]:
class GPT2Model(GPTPreTrainedModel):
  """
  The GPT model returns the final embeddings for each token in a sentence.

  The model consists of:
  1. Embedding layers (used in self.embed).
  2. A stack of n GPT layers (used in self.encode).
  3. A linear transformation layer for the [CLS] token (used in self.forward, as given).
  """

  def __init__(self, config):
    super().__init__(config)
    self.config = config

    # Embedding layers.
    self.word_embedding = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
    self.pos_embedding = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.embed_dropout = nn.Dropout(config.hidden_dropout_prob)

    # Register position_ids (1, len position emb) to buffer because it is a constant.
    position_ids = torch.arange(config.max_position_embeddings).unsqueeze(0)
    self.register_buffer('position_ids', position_ids)

    # GPT-2 layers.
    self.gpt_layers = nn.ModuleList([GPT2Layer(config) for _ in range(config.num_hidden_layers)])

    # [CLS] token transformations.
    self.pooler_dense = nn.Linear(config.hidden_size, config.hidden_size)
    self.pooler_af = nn.Tanh()

    # Final layer norm.
    self.final_layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    self.init_weights()

  def embed(self, input_ids):
    """
    TODO-4: Embedding layer of the GPT-2 model.

    Implementation hints:
    1. Use self.word_embedding to convert input_ids to embeddings.
    2. Generate position ids and convert to embeddings using self.pos_embedding.
    3. Sum token and position embeddings.
    4. Apply self.embed_dropout to the sum.
    """
    ### YOUR CODE HERE
    #get token embeddings
    token_embeddings=self.word_embedding(input_ids)
    batch_size, seq_len = input_ids.size()
    #generate position ids and convert to embeddings
    position_ids = self.position_ids[:, :seq_len]
    position_ids = position_ids.expand(batch_size, seq_len)
    position_embeddings = self.pos_embedding(position_ids)
    #sum token and position embeddings
    sum_of_embeddings=token_embeddings+position_embeddings
    sum_of_embeddings=self.embed_dropout(sum_of_embeddings)
    return sum_of_embeddings

  def encode(self, hidden_states, attention_mask):
    """
    hidden_states: the output from the embedding layer [batch_size, seq_len, hidden_size]
    attention_mask: [batch_size, seq_len]
    """
    # Get the extended attention mask for self-attention.
    # Returns extended_attention_mask of size [batch_size, 1, 1, seq_len].
    # Distinguishes between non-padding tokens (with a value of 0) and padding tokens
    # (with a value of a large negative number).
    extended_attention_mask: torch.Tensor = get_extended_attention_mask(attention_mask, self.dtype)

    # Pass the hidden states through the encoder layers.
    for i, layer_module in enumerate(self.gpt_layers):
      # Feed the encoding from the last bert_layer to the next.
      hidden_states = layer_module(hidden_states, extended_attention_mask)

    return hidden_states

  def forward(self, input_ids, attention_mask):
    """
    input_ids: [batch_size, seq_len], seq_len is the max length of the batch
    attention_mask: same size as input_ids, 1 represents non-padding tokens, 0 represents padding tokens
    """
    # Get the embedding for each input token.
    embedding_output = self.embed(input_ids=input_ids)

    # Feed to a transformer (a stack of GPTLayers).
    sequence_output = self.encode(embedding_output, attention_mask=attention_mask)
    sequence_output = self.final_layer_norm(sequence_output)

    # Get the hidden state of the final token.
    last_non_pad_idx = attention_mask.sum(dim=1) - 1  # Subtract 1 to get last index
    last_token = sequence_output[torch.arange(sequence_output.shape[0]), last_non_pad_idx]

    return {'last_hidden_state': sequence_output, 'last_token': last_token}

  def hidden_state_to_token(self, hidden_state):
    """
    TODO-5: Convert hidden states back to token logits.

    Implementation hints: 
    - GPT-2 uses weight tying with the input word embeddings. 
    - The logits are the dot product between output hidden states and the word embedding weights: hidden_state(s) * E^T
    """
    ### YOUR CODE HERE
    logits = torch.matmul(hidden_state, self.word_embedding.weight.T)
    return logits

  @classmethod
  def from_pretrained(cls, model='gpt2', d=768, l=12, num_heads=12):
    gpt_model = OpenAIGPT2Model.from_pretrained(model).eval()
    our_model = GPT2Model(GPT2Config(hidden_size=d, num_hidden_layers=l,num_attention_heads=num_heads,
                                     intermediate_size=d*3)).eval()

    # Load word and positional embeddings.
    our_model.word_embedding.load_state_dict(gpt_model.wte.state_dict())
    our_model.pos_embedding.load_state_dict(gpt_model.wpe.state_dict())

    for i in range(l):
      l = our_model.gpt_layers[i]
      # Remap the Q,K,V weights from a conv1d to 3 linear projections
      l.self_attention.query.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, :d].T
      l.self_attention.query.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][:d]
      l.self_attention.key.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, d:d*2].T
      l.self_attention.key.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][d:d*2]
      l.self_attention.value.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.weight'][:, d*2:].T
      l.self_attention.value.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_attn.bias'][d*2:]

      # Remap final dense layer in MHA.
      l.attention_dense.weight.data = gpt_model.state_dict()[f'h.{i}.attn.c_proj.weight'].T
      l.attention_dense.bias.data = gpt_model.state_dict()[f'h.{i}.attn.c_proj.bias']

      # Remap attention layer norm.
      l.attention_layer_norm.weight.data = gpt_model.state_dict()[f'h.{i}.ln_1.weight']
      l.attention_layer_norm.bias.data = gpt_model.state_dict()[f'h.{i}.ln_1.bias']

      # Remap post-attention MLP layers.
      l.interm_dense.weight.data = gpt_model.state_dict()[f'h.{i}.mlp.c_fc.weight'].T
      l.interm_dense.bias.data = gpt_model.state_dict()[f'h.{i}.mlp.c_fc.bias']
      l.out_dense.weight.data = gpt_model.state_dict()[f'h.{i}.mlp.c_proj.weight'].T
      l.out_dense.bias.data = gpt_model.state_dict()[f'h.{i}.mlp.c_proj.bias']

      # Remap second layer norm weights.
      l.out_layer_norm.weight.data = gpt_model.state_dict()[f'h.{i}.ln_2.weight']
      l.out_layer_norm.bias.data = gpt_model.state_dict()[f'h.{i}.ln_2.bias']

    # Remap the final layer norm values.
    our_model.final_layer_norm.weight.data = gpt_model.state_dict()['ln_f.weight']
    our_model.final_layer_norm.bias.data = gpt_model.state_dict()['ln_f.bias']

    return our_model

## Adam Optimiser

In [7]:
class AdamW(Optimizer):
    def __init__(
            self,
            params: Iterable[torch.nn.parameter.Parameter],
            lr: float = 1e-3,
            betas: Tuple[float, float] = (0.9, 0.999),
            eps: float = 1e-6,
            weight_decay: float = 0.0,
            correct_bias: bool = True,
    ):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
        super().__init__(params, defaults)

    def step(self, closure: Callable = None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                # State should be stored in this dictionary.
                state = self.state[p]

                # Access hyperparameters from the `group` dictionary.
                lr = group["lr"]
                eps = group["eps"]
                weight_decay = group["weight_decay"]
                correct_bias = group["correct_bias"]
                beta1, beta2 = group["betas"]
                state = self.state[p]
                if len(state) == 0:
                    state["step"] = 0
                    state["exp_avg"] = torch.zeros_like(p.data)
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                
                state["step"] += 1
                t = state["step"]
                
                """
                TODO-6: Implement the AdamW parameter update for this step.

                Implementation hints:
                1. Update biased first moment estimate:
                    m_t = beta1 * m_{t-1} + (1 - beta1) * grad
                2. Update biased second raw moment estimate:
                    v_t = beta2 * v_{t-1} + (1 - beta2) * grad^2
                3. Apply bias correction if correct_bias=True:
                    m_hat = m_t / (1 - beta1^t)
                    v_hat = v_t / (1 - beta2^t)
                4. Compute step size:
                    step_size = lr (or lr / (1 - beta1^t) if bias correction)
                5. Update parameters:
                    p = p - step_size * m_hat / (sqrt(v_hat) + eps)
                6. Apply decoupled weight decay after the parameter update (if weight_decay > 0):
                    p = p - lr * weight_decay * p
                Reference:
                Algorithm 1 in "Adam: A Method for Stochastic Optimization"
                https://arxiv.org/abs/1412.6980
                """
                ### YOUR CODE HERE
                exp_avg = state["exp_avg"]
                exp_avg_sq = state["exp_avg_sq"]

                # update biased first and second moments
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # bias correction
                if correct_bias:
                    m_hat = exp_avg / (1 - beta1 ** t)
                    v_hat = exp_avg_sq / (1 - beta2 ** t)
                else:
                    m_hat = exp_avg
                    v_hat = exp_avg_sq

                # update p
                p.data.addcdiv_(-lr, m_hat, torch.sqrt(v_hat) + eps)

                # apply weight decay
                if weight_decay > 0:
                    p.data.add_(-lr * weight_decay, p.data)
        return loss

## Text Generator

In [8]:
@torch.no_grad()
def generate_gpt2(model, tokenizer, input_ids, max_gen_length=50, device="cuda"):
    """
    Generate text from a GPT-2 model given a single input sequence (greedy decoding).

    Note:
        - Currently only supports batch_size=1 (single input sequence).
        - Using greedy decoding, so each run with the same input produces the same output.
        - Other sampling-based decoding methods (e.g., top-k, top-p, temperature) can introduce randomness and yield different outputs each run.

    Args:
        model: GPT-2 model (pretrained or fine-tuned)
        tokenizer: GPT-2 tokenizer
        input_ids: torch.LongTensor of shape [1, seq_len], input token IDs
        max_gen_length: int, maximum number of tokens to generate
        device: str, "cuda" or "cpu"

    Returns:
        str: Generated text
    """
    model.eval()
    input_ids = input_ids.to(device)  # move input to device
    output_ids = input_ids.clone()

    """
    TODO-8: Greedy next-token generation loop

    Implementation hints:
    Repeat the below steps up to max_gen_length:
    1. Construct an attention mask based on current output_ids (non-pad tokens).
    2. Pass output_ids and attention_mask through the model to get hidden states.
    3. Convert the last hidden state to logits over the vocabulary using model.hidden_state_to_token.
    4. Select the next token using greedy decoding (argmax over logits).
    5. Append the next token to output_ids.
    6. Stop the loop early if the EOS token is generated.
    
    """
    ### YOUR CODE HERE
    for _ in range(max_gen_length):
        attention_mask=torch.ones_like(output_ids)
        hidden_states=model(output_ids,attention_mask) #hidden state is a dictionary w keys last_hidden_state and last_token
        # print(type(hidden_states))
        # print(hidden_states.keys() if hasattr(hidden_states, 'keys') else hidden_states)
        last_hidden = hidden_states['last_hidden_state'][:, -1, :] #shape : [batch_size, sequence_length, hidden_dim]
        logits = model.hidden_state_to_token(last_hidden)
        next_token = torch.argmax(logits,dim=-1).unsqueeze(-1)
        output_ids = torch.cat([output_ids, next_token],dim=-1)
        if (next_token == tokenizer.eos_token_id).any():
            break

    # raise NotImplementedError

    # Decode generated tokens to string
    ids = output_ids[0]
    text = tokenizer.decode(ids, skip_special_tokens=True)

    return text

## Loading NLI Dataset
Where langauges are appended together in a single big dataset for train, dev and test.

In [9]:
def compute_accuracy(preds, labels):
    correct = sum(p.lower().strip() == l.lower().strip() for p, l in zip(preds, labels))
    return correct / len(labels)

def evaluate_gpt2_xnli(model, tokenizer, dataloader, max_gen_length=10, device="cuda"):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for item in tqdm(dataloader, desc="Generating"):
            input_ids = item['input_ids']
            gen_text = generate_gpt2(model, tokenizer, input_ids, max_gen_length=max_gen_length, device=device)
            pred_label = gen_text.split("Label:")[-1].strip()
            all_preds.append(pred_label)
            all_labels.extend(item['label_strs'])
    acc = compute_accuracy(all_preds, all_labels)
    print(f"Evaluation accuracy: {acc*100:.2f}%")
    return acc, all_preds, all_labels

class XNLIDataset(Dataset):
    """
    A PyTorch Dataset for XNLI (Cross-lingual Natural Language Inference) task.

    Supports train, dev, and test splits in a specific language, 
    tokenizes text inputs for GPT-style models, and optionally subsamples the dataset.

    Attributes:
        split (str): Dataset split, one of 'train', 'dev', 'test'.
        lang (str): Language code (e.g., 'en', 'zh').
        tokenizer: A HuggingFace tokenizer to convert text to input IDs.
        max_length (int): Maximum sequence length for tokenization.
        LABEL2ID (dict): Mapping from textual labels to integer IDs.
        ID2LABEL (dict): Reverse mapping from integer IDs to textual labels.
        data (pd.DataFrame): The loaded and preprocessed dataset.
    """
    def __init__(
        self,
        split="train",
        langs=["en"], # Consider list
        train_path_template="XNLI-MT-1.0/multinli/multinli.train.{lang}.tsv",
        test_path="XNLI-1.0/xnli.test.tsv",
        dev_path="XNLI-1.0/xnli.dev.tsv",
        tokenizer=None,
        max_length=1024,
        subset = 1.0  # 0~1
    ):
        self.split = split
        self.langs = langs
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.LABEL2ID = {"entailment": 0, "contradictory": 1, "neutral": 2}
        self.ID2LABEL = {v: k for k, v in self.LABEL2ID.items()}

        if split == "train":
            dfs = []
            for lang in langs:
                path = train_path_template.format(lang=lang)
                df_lang = self.read_xnli_tsv(path, split)
                df_lang = df_lang.dropna(subset=['premise','hypo','label'])
                dfs.append(df_lang)
                
                # de-tokenize for Chinese
                if lang == 'zh': 
                    df_lang['premise'] = df_lang['premise'].str.replace(" ", "")
                    df_lang['hypo'] = df_lang['hypo'].str.replace(" ", "")
                    
                print(f"Loaded {lang}: {len(df_lang)} rows")
                
            # Combine all languages
            df = pd.concat(dfs).reset_index(drop=True)
            print(f"Total number rows: {len(df)}")
            
        elif split in ["dev", "test"]:
            path = test_path if split=="test" else dev_path
            df = self.read_xnli_tsv(path, split)
            # Obtain and append all languages in the given list
            df = df[df['language'].isin(langs)].copy()
            keep_cols = ['sentence1', 'sentence2', 'gold_label']
            df = df[keep_cols].dropna()
            df.rename(columns={'sentence1':'premise','sentence2':'hypo','gold_label':'label'}, inplace=True)
            df['label'] = df['label'].replace({'contradiction': 'contradictory'})
        else:
            raise ValueError("split must be one of ['train','dev','test']")
        
        original_num = len(df)
        if subset < 1.0:
            # Randomly sample dataset, especially for a mixture of languages
            n = max(1, int(len(df) * subset))
            df = df.sample(n=n, random_state=42).reset_index(drop=True)

        subset_num = len(df)

        self.data = df.reset_index(drop=True)
        print(f"Dataset initialized: split='{split}', lang='{langs}', total={original_num}, subset={subset}, subset_count={subset_num}")

    def read_xnli_tsv(self, path, split):
        """
        Read an XNLI TSV file and return it as a pandas DataFrame.

        Args:
            path (str): Path to the TSV file.
            split (str): One of "train", "dev", "test" indicating the dataset split.

        Returns:
            pd.DataFrame: The dataset as a DataFrame with appropriate columns.
        """
        if split == "train":
            with open(path, "r", encoding="utf-8") as f:
                lines = f.read().splitlines()
            header = lines[0].split("\t")
            data = []
            for i, line in enumerate(lines[1:], start=2):
                parts = line.split("\t")
                if len(parts) == len(header):
                    data.append(parts)
                else:
                    print(f"skip row {i}: {len(parts)} cols → {parts[:2]}")
        else:
            with open(path, "r", encoding="utf-8") as f:
                reader = csv.reader(f, delimiter="\t")
                rows = list(reader)
            header = rows[0]
            expected_cols = len(header)
            data = []
            for i, row in enumerate(rows[1:], start=2):
                if len(row) == expected_cols:
                    data.append(row)
                else:
                    print(f"skip row {i}: {len(row)} cols → {row[:2]}")
        return pd.DataFrame(data, columns=header)

    def __len__(self):
        """Return the number of examples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieve a single example by index and tokenize it.

        For training split:
            - Constructs the input as "Premise: ... Hypothesis: ... Label: ..."
            - Tokenizes the full input.
            - Masks the prefix tokens in the labels with -100 for GPT loss computation.

        For dev/test split:
            - Constructs the input without label as "Premise: ... Hypothesis: ... Label:"

        Returns:
            dict: Contains 'input_ids', 'attention_mask', 'labels' (train only), 'label_str'
        """
        row = self.data.iloc[idx]
        premise = row['premise']
        hypo = row['hypo']
        label = row['label']

        if self.split == "train":
            prefix = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            full_text = prefix + str(self.LABEL2ID[label])
            tokenized = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

            prefix_ids = self.tokenizer(prefix).input_ids
            labels_ids = tokenized['input_ids'].clone()
            labels_ids[:len(prefix_ids)] = -100 # Masks the prefix tokens in the labels with -100 for GPT loss computation.
            tokenized['labels'] = labels_ids
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized
        else:
            text = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            tokenized = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized

    @staticmethod
    def collate_fn(batch):
        """
        Collate a batch of examples into padded tensors.

        Pads 'input_ids' and 'attention_mask' to the max length in the batch.
        Pads 'labels' with -100 if present.
        Collects 'label_str' for reference.

        Returns:
            dict: Padded tensors and label strings for the batch.
        """
        input_ids = torch.nn.utils.rnn.pad_sequence(
            [b['input_ids'] for b in batch],
            batch_first=True,
            padding_value=0
        )
        attention_mask = torch.nn.utils.rnn.pad_sequence(
            [b['attention_mask'] for b in batch],
            batch_first=True,
            padding_value=0
        )

        if 'labels' in batch[0]:
            labels = torch.nn.utils.rnn.pad_sequence(
                [b['labels'] for b in batch],
                batch_first=True,
                padding_value=-100
            )
        else:
            labels = None

        label_strs = [b['label_str'] for b in batch]

        out = {"input_ids": input_ids, "attention_mask": attention_mask, "label_strs": label_strs}
        if labels is not None:
            out["labels"] = labels
        return out

## Fine-tune GPT-2 (all language)

Guidance: Load the pretrained GPT-2 (again, not the ones fine-tuned on English NLI) along with the training data for all target languages, including English. 

For non-English languages, select those that performed well in the zero-shot cross-lingual transfer and fertility evaluation. It depends on you how many languages to include. 

Fine-tune a single model on this combined multilingual dataset. Afterwards, compare this model with the per-language fine-tuned models and the zero-shot cross-lingual transfer results.

### Generic retraining pipeline
1. Load dataset from XNLIDataset and split it into test and train.
2. Use DataLoader to load it up.
3. Use Adam optimiser from the function AdamW(Optimizer).
4. Build the training loop (intitialise weights, train, get loss, gradietn descent, early stopping based on dev_dataset accuracy/results).
5. Save the best checkpoint per langauge
6. Evaluate via test_set
7. Then comapre with its zero-shot counterpart.

In [10]:
from transformers import get_linear_schedule_with_warmup

# Adjust from 0 to 1, but consider 0.1 for rough testing of pipeline
TRAIN_SUBSET = 1 #0.001 for testing, 1 for actual fine-tuning
DEV_SUBSET = 1
TEST_SUBSET = 1

# Hyperparamter of gpt2 fine-tuning
EPOCHS = 5 # 2 for testing, 1 for actual fine-tuning
BATCH_SIZE = 8
LR = 2e-5
WEIGHT_DECAY = 0.01
WARMUP_RATIO=0.1
CORRECT_BIAS = True
SAVE_DIR = "best_model"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
VOCAB_SIZE = tokenizer.vocab_size


def split_dataset(langs, tokenizer):
    train_dataset = XNLIDataset(
        split="train",
        langs=langs,
        tokenizer=tokenizer,
        subset=TRAIN_SUBSET
    )

    dev_dataset = XNLIDataset(
        split="dev",
        langs=langs,
        tokenizer=tokenizer,
        subset=DEV_SUBSET
    )

    test_dataset = XNLIDataset(
        split="test",
        langs=langs,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
    
    return train_dataset, dev_dataset, test_dataset


def full_finetuning_pipeline(train_dataset, dev_dataset, lang, start_epoch=0):
    # Track best dev accuracy for model saving
    # This only works for epoch > 1 
    best_dev_acc = 0.0
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    # Create DataLoaders for training and validation datasets
    train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=XNLIDataset.collate_fn)
    dev_loader = DataLoader(dev_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)

    # Scheduler is on by default
    scheduler = None
    total_steps = len(train_loader) * EPOCHS
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    # Initialize optimizer & scheduler
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, correct_bias=CORRECT_BIAS)
    scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    
    # Load from checkpoint if possible
    if start_epoch > 0:
        optimizer_ckpt = f"{SAVE_DIR}/optimizer_{lang}_epoch_{start_epoch}.pt"
        scheduler_ckpt = f"{SAVE_DIR}/scheduler_{lang}_epoch_{start_epoch}.pt"
        if os.path.exists(optimizer_ckpt) and os.path.exists(scheduler_ckpt):
            print(f"Loading optimizer and scheduler from epoch {start_epoch} checkpoint...")
            optimizer.load_state_dict(torch.load(optimizer_chkpt))
            scheduler.load_state_dict(torch.load(scheduler_ckpt))
    else:
        print("Checkpoint not found. Starting optimizer and scheduler from scratch.")
        
    # Include early stopping
    patience = 2
    epochs_without_improvement = 0
    
    # Track training progress
    global_train_losses = []
    total_train_loss = 0.0
    total_train_steps = 0

    # Training loop
    for epoch in range(start_epoch, EPOCHS):
        print(f"Epoch {epoch+1}/{EPOCHS}")
        model.train()
        # Iterate over batches
        loop = tqdm(train_loader, desc="Training")
        for batch in loop:
            input_ids = batch["input_ids"].to(DEVICE)        # [B, seq_len]
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch.get("labels").to(DEVICE)                    # [B, seq_len]

            optimizer.zero_grad()

            hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']  # [B, seq_len, hidden]

            """
            Reused from to-do 9.

            Implementation hints:
            1. Convert hidden states to logits over the vocabulary using model.hidden_state_to_token.
            2. Shift logits and labels for next-token prediction to align each prediction with the correct next token.
            3. Compute the cross-entropy loss, making sure positions with label=-100 are ignored.
            4. Backpropagate and update model parameters.
            """

            ### YOUR CODE HERE
            logits = model.hidden_state_to_token(hidden_states)
            shifted_logits = logits[:, :-1, :] #take all but last token cause mismatch right now -> output pred 0, pred 1, pred 2
            shifted_labels = labels[:,1:] #take all but first token                                labels word1 , word 2, word 3
            loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)                                  #but we want pred 0 to predict word 1, pred 1 to predict word 2, pred 2 to predict word 3, so remove first label so labels look like [word2,word3..] and predictions remove last since there is no more word after last word.
            loss = loss_fn(shifted_logits.reshape(-1,logits.size(-1)), shifted_labels.reshape(-1))  # flatten to [B*(seq_len-1), vocab_size] since N= B*(seq_len-1) for logits and [B*(seq_len-1)] for labels
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # raise NotImplementedError     
            total_train_loss += loss.item()
            total_train_steps += 1
            global_train_avg_loss = total_train_loss / total_train_steps
            global_train_losses.append(global_train_avg_loss)

            loop.set_postfix({'avg_loss': f"{global_train_avg_loss:.4f}"})

        print(f"Epoch {epoch+1} finished | Global Avg Loss: {global_train_avg_loss:.4f}")
        
        # Save model at every epoch
        torch.save(model.state_dict(), f"{SAVE_DIR}/model_multi_epoch_{epoch}.pt")
        # Save optimiser at every epoch
        torch.save(optimizer.state_dict(), f"{SAVE_DIR}/optimizer_{lang}_epoch_{epoch}.pt")
        # Save scheduler as well
        torch.save(scheduler.state_dict(), f"{SAVE_DIR}/scheduler_{lang}_epoch_{epoch}.pt")

        acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, dev_loader, max_gen_length=1, device=DEVICE)

        
        # Save best model
        if acc > best_dev_acc:
            best_dev_acc = acc
        # Or trigger early stopping
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered.")
                return
        
        
def compute_fertility(dataset, tokenizer):
    total_words = 0
    total_tokens = 0
    samples = len(dataset)
    
    for i in tqdm(range(samples), desc="Computing fertility"):
        row = dataset.data.iloc[i]
        for sent in [row['premise'], row['hypo']]:
            words = sent.strip().split()  # crude word estimate
            tokens = tokenizer.tokenize(sent)
            total_words += len(words)
            total_tokens += len(tokens)
    
    fertility = total_tokens / total_words if total_words > 0 else 0.0
    return fertility


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Note: Uncomment certain lines of code for fine-tuning. It's the same code cell used for pre-training so the logs were lost but I do save the logs in a text file 😭.

```shell
Loaded en: 392702 rows
Loaded vi: 392702 rows
Loaded zh: 392702 rows
Loaded es: 392702 rows
Total number rows: 1570808
Dataset initialized: split='train', lang='['en', 'vi', 'zh', 'es']', total=1570808, subset=1, subset_count=1570808
Dataset initialized: split='dev', lang='['en', 'vi', 'zh', 'es']', total=9960, subset=1, subset_count=9960
Dataset initialized: split='test', lang='['en', 'vi', 'zh', 'es']', total=20040, subset=1, subset_count=20040
Epoch 1/5
Training:   0%|          | 320/196351 [00:42<8:06:57,  6.71it/s, avg_loss=9.3432]Token indices sequence length is longer than the specified maximum sequence length for this model (1737 > 1024). Running this sequence through the model will result in indexing errors
Training: 100%|██████████| 196351/196351 [7:12:53<00:00,  7.56it/s, avg_loss=0.8379]  
Epoch 1 finished | Global Avg Loss: 0.8379
Generating: 100%|██████████| 9960/9960 [01:32<00:00, 107.72it/s]
Evaluation accuracy: 68.61%
Epoch 2/5
Training: 100%|██████████| 196351/196351 [7:09:40<00:00,  7.62it/s, avg_loss=0.7056]3]
Epoch 3 finished | Global Avg Loss: 0.7056
Generating: 100%|██████████| 9960/9960 [01:33<00:00, 106.30it/s]
Evaluation accuracy: 72.81%
Early stopping triggered.
Epoch 4/5
Training:   2%|▏         | 3554/196351 [07:48<6:56:01,  7.72it/s, avg_loss=0.7047] 
```

In [ ]:
# Main loop
import time

# Empty cache before starting
torch.cuda.empty_cache()


chosen_langs = ["en", "vi", "zh", "es"]


# Load a pretrained GPT-2 model with official weights
model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

# Then continue from checkpoint if training was disrupted
model.load_state_dict(torch.load("./best_model/model_multi_epoch_2.pt"))

# Uncomment this for training
# Split train, val and test
# train_dataset, dev_dataset, test_dataset = split_dataset(chosen_langs, tokenizer)
# print(train_dataset.data.head()) # Double check if it's a mixed language

# Full finetunign pipeline, refer to code above for params\
# full_finetuning_pipeline(train_dataset, dev_dataset, lang=chosen_langs, start_epoch=4)

# Uncomment this for testing
model.eval()
test_dataset = XNLIDataset(
        split="test",
        langs=chosen_langs,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)


start_time = time.time()
acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
end_time = time.time()
time_taken = end_time - start_time

print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")    
    

Dataset initialized: split='test', lang='['en', 'vi', 'zh', 'es']', total=20040, subset=1, subset_count=20040


Generating: 100%|██████████| 20040/20040 [03:13<00:00, 103.68it/s]

Evaluation accuracy: 72.56%
Training took 3.22 minutes for all 20040 samples


Comparing with another weight from earlier epochs.

In [13]:
# Main loop
import time

# Empty cache before starting
torch.cuda.empty_cache()


chosen_langs = ["en", "vi", "zh", "es"]


# Load a pretrained GPT-2 model with official weights
model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

# Then continue from checkpoint if training was disrupted
model.load_state_dict(torch.load("./best_model/model_multi_epoch_4.pt"))


# Uncomment this for testing
model.eval()
test_dataset = XNLIDataset(
        split="test",
        langs=chosen_langs,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)


start_time = time.time()
acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
end_time = time.time()
time_taken = end_time - start_time

print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")    
    

Dataset initialized: split='test', lang='['en', 'vi', 'zh', 'es']', total=20040, subset=1, subset_count=20040


Generating: 100%|██████████| 20040/20040 [03:11<00:00, 104.44it/s]

Evaluation accuracy: 70.97%
Training took 3.20 minutes for all 20040 samples


## Per language testing of multilingual weight

In [27]:
import time

torch.cuda.empty_cache()


# Load a pretrained GPT-2 model with official weights
model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

# Then continue from checkpoint if training was disrupted
model.load_state_dict(torch.load("./best_model/model_multi_epoch_2.pt"))
model.eval()

for lang in ["en", "vi", "zh", "es"]:
    chosen_langs = [lang]
    
    test_dataset = XNLIDataset(
            split="test",
            langs=chosen_langs,
            tokenizer=tokenizer,
            subset=TEST_SUBSET
        )
                    
    test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)

    start_time = time.time()
    acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
    end_time = time.time()
    time_taken = end_time - start_time

    print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")

Dataset initialized: split='test', lang='['en']', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:45<00:00, 109.43it/s]


Evaluation accuracy: 83.31%
Training took 0.76 minutes for all 5010 samples
Dataset initialized: split='test', lang='['vi']', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:52<00:00, 95.18it/s] 


Evaluation accuracy: 68.34%
Training took 0.88 minutes for all 5010 samples
Dataset initialized: split='test', lang='['zh']', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:49<00:00, 101.47it/s]


Evaluation accuracy: 66.13%
Training took 0.82 minutes for all 5010 samples
Dataset initialized: split='test', lang='['es']', total=5010, subset=1, subset_count=5010


Generating: 100%|██████████| 5010/5010 [00:47<00:00, 104.40it/s]

Evaluation accuracy: 72.46%
Training took 0.80 minutes for all 5010 samples


## Code-switched test with custom CSDataset

In [11]:
class CSDataset(Dataset):
    def __init__(
        self,
        split="test",
        lang="zh",
        test_path="cs_dataset/cs_{lang}_test.tsv",
        tokenizer=None,
        max_length=1024,
        subset = 1.0  # 0~1
    ):
        self.split = split
        self.lang = lang
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.LABEL2ID = {"entailment": 0, "contradictory": 1, "neutral": 2}
        self.ID2LABEL = {v: k for k, v in self.LABEL2ID.items()}

        if split=="test":
            path = test_path.format(lang=self.lang)
            df = self.read_dataset_tsv(path, split)
            keep_cols = ['sentence1', 'sentence2', 'gold_label']
            df = df[keep_cols].dropna()
            df.rename(columns={'sentence1':'premise','sentence2':'hypo','gold_label':'label'}, inplace=True)
            df['label'] = df['label'].replace({'contradiction': 'contradictory'})
        else:
            raise ValueError("split must be one of ['train','dev','test']")
        
        original_num = len(df)
        if subset < 1.0:
            n = max(1, int(len(df) * subset))
            df = df.iloc[:n].reset_index(drop=True)
        subset_num = len(df)

        self.data = df.reset_index(drop=True)
        print(f"Dataset initialized: split='{split}', lang='{lang}', total={original_num}, subset={subset}, subset_count={subset_num}")

    def read_dataset_tsv(self, path, split):
        if split == "train":
            with open(path, "r", encoding="utf-8") as f:
                lines = f.read().splitlines()
            header = lines[0].split("\t")
            data = []
            for i, line in enumerate(lines[1:], start=2):
                parts = line.split("\t")
                if len(parts) == len(header):
                    data.append(parts)
                else:
                    print(f"skip row {i}: {len(parts)} cols → {parts[:2]}")
        else:
            with open(path, "r", encoding="utf-8") as f:
                reader = csv.reader(f, delimiter="\t")
                rows = list(reader)
            header = rows[0]
            expected_cols = len(header)
            data = []
            for i, row in enumerate(rows[1:], start=2):
                if len(row) == expected_cols:
                    data.append(row)
                else:
                    print(f"skip row {i}: {len(row)} cols → {row[:2]}")
        return pd.DataFrame(data, columns=header)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        premise = row['premise']
        hypo = row['hypo']
        label = row['label']
        if self.lang == 'zh': # de-tokenize for Chinese
            premise = premise.replace(" ", "")
            hypo = hypo.replace(" ", "")

        if self.split == "train":
            prefix = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            full_text = prefix + str(self.LABEL2ID[label])
            tokenized = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}

            prefix_ids = self.tokenizer(prefix).input_ids
            labels_ids = tokenized['input_ids'].clone()
            labels_ids[:len(prefix_ids)] = -100 # Masks the prefix tokens in the labels with -100 for GPT loss computation.
            tokenized['labels'] = labels_ids
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized
        else:
            text = f"Premise: {premise}\nHypothesis: {hypo}\nLabel:"
            tokenized = self.tokenizer(
                text,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt"
            )
            tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}
            tokenized['label_str'] = str(self.LABEL2ID[label])
            return tokenized

In [12]:
import time

torch.cuda.empty_cache()


# Load a pretrained GPT-2 model with official weights
model = GPT2Model.from_pretrained("gpt2").to(DEVICE)

# Then continue from checkpoint if training was disrupted
model.load_state_dict(torch.load("./best_model/model_multi_epoch_2.pt"))
model.eval()

for lang in ["vi", "zh", "es"]:
    
    test_dataset = CSDataset(
        split="test",
        lang=lang,
        tokenizer=tokenizer,
        subset=TEST_SUBSET
    )
                    
    test_loader = DataLoader(test_dataset,shuffle=False,collate_fn=XNLIDataset.collate_fn)

    start_time = time.time()
    acc, all_preds, all_labels = evaluate_gpt2_xnli(model, tokenizer, test_loader, max_gen_length=1, device=DEVICE)
    end_time = time.time()
    time_taken = end_time - start_time

    print(f"Training took {time_taken/60:.2f} minutes for all {len(test_dataset)} samples")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Dataset initialized: split='test', lang='vi', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:08<00:00, 121.19it/s]


Evaluation accuracy: 66.70%
Training took 0.14 minutes for all 1000 samples
Dataset initialized: split='test', lang='zh', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:09<00:00, 109.76it/s]


Evaluation accuracy: 57.00%
Training took 0.15 minutes for all 1000 samples
Dataset initialized: split='test', lang='es', total=1000, subset=1, subset_count=1000


Generating: 100%|██████████| 1000/1000 [00:07<00:00, 129.35it/s]

Evaluation accuracy: 68.20%
Training took 0.13 minutes for all 1000 samples


## Checking model params size

In [13]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+-------------------------------------------+------------+
|                  Modules                  | Parameters |
+-------------------------------------------+------------+
|           word_embedding.weight           |  38597376  |
|            pos_embedding.weight           |   786432   |
|  gpt_layers.0.self_attention.query.weight |   589824   |
|   gpt_layers.0.self_attention.query.bias  |    768     |
|   gpt_layers.0.self_attention.key.weight  |   589824   |
|    gpt_layers.0.self_attention.key.bias   |    768     |
|  gpt_layers.0.self_attention.value.weight |   589824   |
|   gpt_layers.0.self_attention.value.bias  |    768     |
|    gpt_layers.0.attention_dense.weight    |   589824   |
|     gpt_layers.0.attention_dense.bias     |    768     |
|  gpt_layers.0.attention_layer_norm.weight |    768     |
|   gpt_layers.0.attention_layer_norm.bias  |    768     |
|      gpt_layers.0.interm_dense.weight     |  2359296   |
|       gpt_layers.0.interm_dense.bias      |    3072   

125030400